# This is a nice visualization tool for the trigger turn-ons

In [1]:
import ROOT
ROOT.enableJSVis()
from ROOT import TFile, TCanvas, TGraph, TF1, TMath, TLine
from ROOT import gBenchmark, gStyle, gROOT
from ROOT import kBlack, kBlue, kRed, kCyan, kMagenta
from array import array
gStyle.SetOptStat(0)

Welcome to JupyROOT 6.14/00


## Here we import the latest data

In [2]:
DTMode = True
if DTMode:
    file = TFile("output-DATA-2a.root","READ")
else:
    file = TFile("output-MC-2a.root","READ")
#file.cd("Standard/Eta_0.0-1.3/")
#file.ls()

In [3]:
Year = 16
if Year==16:
    triggers = ["0","40","60","80","140","200","260","320","400","450"]
else:
    triggers = ["0","40","60","80","140","200","260","320","400","450","500"]
hpts = []
hpt0s = []
for trg in triggers:
    hpts.append(file.Get("Standard/Eta_0.0-1.3/jt"+trg+"/hpt"))
    hpt0s.append(file.Get("Standard/Eta_0.0-1.3/jt"+trg+"/hpt0"))

# Choose parameters and optionally, look at the data

In [4]:
Trg = 3
Preview = False
RunParams = []
if Year==16:
    if Trg==1: # HLTPFJet40
        RunParams = [0.95,1.05,15,90,-0.1,0.1,1,1.025 if DTMode else 1.0]
    if Trg==2: # HLTPFJet60
        RunParams = [0.95,1.05,40,130,-0.1,0.1,1,1.0 if DTMode else 1.0]
    if Trg==3: # HLTPFJet80
        RunParams = [0.95,1.05,62,154,-0.1,0.1,1,1.0 if DTMode else 1.0]
    if Trg==4: # HLTPFJet140
        RunParams = [0.95,1.05,80,245,-0.1,0.1,1,0.99 if DTMode else 1.0]
    if Trg==5: # HLTPFJet200
        RunParams = [0.95,1.05,145,300,-0.1,0.1,1,1 if DTMode else 1.0]
    if Trg==6: # HLTPFJet260
        RunParams = [0.95,1.05,205,362,-0.1,0.1,1,0.99 if DTMode else 1.0]
    if Trg==7: # HLTPFJet320
        RunParams = [0.95,1.05,266,430,-0.1,0.1,1,1 if DTMode else 1.0]
    if Trg==8: # HLTPFJet400
        RunParams = [0.95,1.05,330,548,-0.1,0.1,1,1 if DTMode else 1.0]
    if Trg==9: # HLTPFJet450
        RunParams = [0.95,1.05,408,686,0.1,0.22,1,1 if DTMode else 1.0]
ymin = RunParams[0]
ymax = RunParams[1]
xmin = RunParams[2]
xmax = RunParams[3]
cmin = RunParams[4]
cmax = RunParams[5]
shift = RunParams[6]
scale = RunParams[7]

Turn = hpts[Trg].Clone("hpt_"+triggers[Trg])
Turn.Divide(hpts[Trg-shift])
Turn.Scale(1.0/scale)
Turn_0 = hpt0s[Trg].Clone("hpt0_"+triggers[Trg])
Turn_0.Divide(hpt0s[Trg-shift])
Turn_0.Scale(1.0/scale)

if Preview:
    c = TCanvas("c","c",900,800)
    Turn_0.SetAxisRange(xmin,xmax,"X")
    Turn_0.SetAxisRange(0,1.2,"Y")
    Turn_0.Draw()
    Turn.Draw("SAME")
    c.Draw()
    c.SetLogx()
    c.Update()

## Fit type "square fraction": $\frac{p_0}{2}\left(\frac{x-p_2}{\sqrt{p_1^2+(x-p_2)^2}}+1\right) + p_3$

In [5]:
def Fun3(x,par):
    return par[0]*((x[0]-par[2])/pow(pow(par[1],2)+pow((x[0]-par[2]),2),0.5)+1)/2.0+par[3]
#def Fun3(x,par):
#    return par[0]/(1+TMath.Exp(-par[1]*(x[0]-par[2])))+par[3]

fit3 = TF1("squarefrac",Fun3,xmin,xmax,4)
fit3.SetParLimits(0,ymin-cmin,ymax-cmin);
fit3.SetParLimits(1,0.0001,1000);
fit3.SetParLimits(2,xmin,xmax);
fit3.SetParLimits(3,cmin,cmax);
fit3.SetParameters((ymin+ymax)/2.0,20,(xmin+xmax)/2.0,(cmin+cmax)/2.0);

c3 = TCanvas("c3","c3",900,800)
Turn_0.Fit(fit3,"M","SAME",xmin,xmax)
Turn_0.SetAxisRange(xmin,xmax,"X")
Turn_0.SetAxisRange(0,1.2,"Y")
Turn_0.Draw()
Turn.Draw("SAME")
c3.Draw()
c3.SetLogx()

p3s = [fit3.GetParameter(0),fit3.GetParameter(1),fit3.GetParameter(2),fit3.GetParameter(3)]
turnpoint95 = p3s[2]+p3s[1]/TMath.Sqrt((TMath.Power(0.9-0.1*p3s[3]/p3s[0],-2)-1))
turnval95 = fit3.Eval(turnpoint95)
turnpoint99 = p3s[2]+p3s[1]/TMath.Sqrt((TMath.Power(0.98-0.02*p3s[3]/p3s[0],-2)-1))
turnval99 = fit3.Eval(turnpoint99)
maxval = p3s[0]+p3s[3]
print(turnpoint95,turnval95,turnval95/maxval)
print(turnpoint99,turnval99,turnval99/maxval)
l95 = TLine(turnpoint95,0,turnpoint95,1)
l95.SetLineColor(kMagenta)
l95.Draw()
l99 = TLine(turnpoint99,0,turnpoint99,1)
l99.SetLineColor(kCyan)
l99.Draw()
l1 = TLine(xmin,1,xmax,1)
l1.SetLineColor(kBlack)
l1.Draw()

c3.Update()

121.56400087515986 1.0924999999999483 0.9500000000000001
129.84901785032386 1.138499999999946 0.9900000000000001
 FCN=31709.8 FROM HESSE     STATUS=OK             27 CALLS         288 TOTAL
                     EDM=2.68955e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.05000e+00   3.52143e-05   3.26355e-03** at limit **
   2  p1           3.02169e+00   5.40018e-02   4.78073e-05   1.38648e-02
   3  p2           1.15651e+02   4.36738e-02   4.67957e-05   7.24651e-03
   4  p3           1.00000e-01   1.73021e-04   5.11357e-03** at limit **


Info in ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 


## We get pT = 509 GeV for HLT_PFJet450!

## Notes

We have fitted a function to the fraction of hpt0 (leading jet inclusive histogram). The fraction is provided by comparing adjacent triggers (e.g. PFJet60 to PFJet40). The histogram hpt (inclusive histogram for all jets) is more stable, but more scarce, and hence hpt0 is a better fitting target. Notably, hpt0 tends to  overshoot slightly, but the fits are still quite good.

Sometimes the histogram hpt does not stabilize at 1.0 on high pT values, and hence we sometimes scale the fraction histograms slightly. Two points from the fit are given: 95% of the maximum value and 99% of the maximum value. A classical turn-on point with these fits is generally reached at the 95% point.

In [6]:
len(hpts)

10

In [7]:
uusio=TCanvas("cu","cu",900,800)
hpts[9].Draw()
uusio.Draw()